In [ ]:
import numpy as onp
import jax.numpy as np

from jax import grad, jit, random

key = random.PRNGKey(1)

In [ ]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([0, 1, 1, 0])
learning_rate = 1.0

In [ ]:
def sigmoid(x):
    return 1 / ( 1 + np.exp(-x))

def neural_network(num_layers, key):
    network_params = {}
    keys = random.split(key, num_layers)
    
    #layer 1
    w_key, b_key = random.split(keys[0])
    network_params['w_1'] = random.normal(w_key, (3, 2))
    network_params['b_1'] = random.normal(b_key, (3, ))
    w_key, b_key = random.split(keys[1])
    network_params['w_2'] = random.normal(w_key, (3,))
    network_params['b_2'] = random.normal(w_key, (1,))
    
    return network_params       

In [7]:
params = neural_network(2, key)

jit_sigmoid = jit(sigmoid)

def fwd_pass(params, x):
        
    inp_to_h1 =sigmoid(np.dot(params['w_1'], x) + params['b_1'])
    out =sigmoid(np.dot(params['w_2'], inp_to_h1) + params['b_2'])
    
    return out

fwd_pass = jit(fwd_pass)

def loss(params, x, y):
    y_pred = fwd_pass(params, x)
    
    val = - (y * np.log(y_pred) + (1 - y)*np.log(1 - y_pred))
    
    return val[0]

loss = jit(loss)


loss_grad = jit(grad(loss))
    

In [8]:
for i in range(300):
    for x, y in zip(X, Y):
        grads = loss_grad(params, x, y)
        #print(params)
        for key in params.keys():
            params[key] = params[key] - learning_rate * grads[key]            

predictions = [f"X: {x} Y: {int(fwd_pass(params, x) > 0.5)}" for x in X]
print(predictions)      
    

['X: [0 0] Y: 0', 'X: [0 1] Y: 1', 'X: [1 0] Y: 1', 'X: [1 1] Y: 0']
